## 講習3 --- 星の測光  

点光源の明るさを測定してみます。ここでは、IRAFのAPPHOTを用いて、アパーチャ測光を行います。  


### 星をひとつだけ選んで測光してみる  

講習2で処理をした、idf0011.fitsの真ん中あたりの星の明るさを測ってみましょう。  

#### 準備 
- 星の位置を求めておく
- 星のサイズ(fwhm)を求めておく
- 背景のレベルとばらつきをもとめておく

In [1]:
from pyraf import iraf

In [2]:
iraf.display('idfs0011.fits')

z1=-69.72522 z2=121.3847


In [3]:
iraf.imexam()

#   COL    LINE   COORDINATES
#     R    MAG    FLUX     SKY    PEAK    E   PA BETA ENCLOSED   MOFFAT DIRECT
 515.88  508.70 515.88 508.70
   8.15  12.54  95977.      0.   8892. 0.16   45 3.50     2.74     2.79   2.72
#            SECTION     NPIX     MEAN   MEDIAN   STDDEV      MIN      MAX
   [546:550,495:499]       25   -3.346       0.    18.52   -41.15    36.59
   [545:549,532:536]       25    6.938       0.    27.31   -33.88     63.1
   [493:497,540:544]       25    1.546       0.    28.95   -88.14    60.29
   [475:479,497:501]       25 -0.09245       0.    18.24   -55.12    39.83


星の位置は(x, y) = (516, 509)で、fwhm=2.8、背景のレベルとばらつきは、0と20としておきましょう。 

#### パラメータの設定  

apphotモジュールをimportします。

In [4]:
from iraf import apphot

~/iraf/login.cl の記載内容によっては、
``` python 
from iraf import noao
from iraf import digiphot
``` 
も必要かもしれません。 

次に、測光に必要なパラメータを設定します。  

In [5]:
med = 0.  #  背景レベルとばらつき、fwhm
std = 20.
fwhm = 2.8

iraf.apphot.unlearn()  #  デフォルト値に戻しておく

iraf.apphot.datapars.datamax = 20000  #  検出器に特有な値
iraf.apphot.datapars.readnoise = 30 #  検出器に特有な値
iraf.apphot.datapars.epadu = 5 #  検出器に特有な値
iraf.apphot.datapars.itime = 5  # 積分時間 

iraf.apphot.phot.interactive = 'no'  
iraf.apphot.phot.verify = 'no'
iraf.apphot.phot.verbose = 'no'
iraf.apphot.photpars.zmag = 21  # 等級のゼロ点

iraf.apphot.datapars.fwhmpsf = fwhm   #  fwhmで決まるパラメータ
iraf.apphot.centerpars.cbox = max(5.0, fwhm)
iraf.apphot.fitskypars.annulus = 3 * fwhm
iraf.apphot.photpars.apertures = 3 * fwhm
iraf.apphot.fitskypars.dannulus = 5.

iraf.apphot.datapars.sigma = std  #  背景のレベルとばらつきで決まるパラメータ
iraf.apphot.datapars.datamin = med - 5 * std

星のおよその位置をテキストファイルに書き込んでおきます。これはテキストエディタで手動で書き込んでおきましょう。

In [6]:
cat coo.txt   #  catはファイルの中身を見るlinuxコマンド 

516 509


この座標ファイルを、**iraf.phot()**に読み込ませます。

In [7]:
iraf.phot('idfs0011.fits', coords='coo.txt', output='out0.mag')

最初の引数は、測光する対象のFITSファイル名です。coords=で読み込ませる座標ファイルの名前、output=で結果を書き出すファイル名を指定します。  

結果のファイルは、iraf.phot()固有の形式で書き出されています。

In [8]:
#cat 'out0.mag'   # githubでは長くなるのでコメントアウトしておきます

通常、必要なのは、xcenter, ycenter, mag, merr です。**iraf.txdump()**を使って、それらだけを抜き出します。

In [8]:
iraf.txdump('out0.mag', fields='xc,yc,mag,merr')

515.881  508.628  10.301  0.005


Stdout=でファイル名を指定すれば、そこに書き出してくれます。

In [9]:
iraf.txdump('out0.mag', fields='xc,yc,mag,merr', Stdout='result0.txt')

これで、ひとつだけ選んだ星の機械等級がもとまりました。  

### 視野の星を検出させて測光してみる

この視野の他の星の明るさも測りたいときには、どうするのでしょうか。まさか、全ての星の位置について手動で座標ファイルを作成するわけにはいきません。**iraf.daofind()**を使います。背景の'ばらつき'よりも何倍明るい星だけを見つけろ、という指示を与えます。  

上の測光に必要なパラメータ設定に加えて、次も設定してやります。

In [10]:
iraf.apphot.findpars.threshold = 7  #  7シグマ以上のものを検出せよ 
iraf.apphot.findpars.sharphi = 0.8  # 星っぽくないものを除くため

iraf.apphot.daofind.output = 'out1.coo'  #  daofindの出力には、よく.cooという拡張子が使われる
iraf.apphot.daofind.interac = 'no'
iraf.apphot.daofind.verify = 'no'

In [11]:
iraf.daofind('idfs0011.fits')

In [12]:
#cat 'out1.coo'  # githubでは長くなるのでコメントアウトしておきます

60個くらい星が検出されたようです。  
先ほどと同様に、このファイルをiraf.phot()のcoords=で指定してやり、今度は結果をout1.magに出しましょう。

In [13]:
iraf.phot('idfs0011.fits', coords='out1.coo', output='out1.mag')

In [14]:
iraf.txdump('out1.mag', fields='xc,yc,mag,merr')

312.046  30.898  13.601  0.085
785.510  63.492  11.574  0.014
640.120  75.714  14.393  0.178
644.934  105.562  14.427  0.187
250.493  113.488  11.479  0.013
344.480  116.890  12.707  0.040
603.322  119.941  12.490  0.034
222.478  129.444  12.646  0.038
630.258  176.196  14.349  0.173
713.843  188.084  13.186  0.058
502.040  207.513  12.989  0.047
794.280  208.412  14.513  0.197
948.199  235.360  14.279  0.170
174.436  237.846  13.545  0.091
35.209  238.813  11.926  0.020
38.877  244.428  12.141  0.023
673.529  250.964  13.739  0.100
528.589  265.680  14.405  0.193
598.709  265.575  13.992  0.119
646.488  270.671  11.053  0.008
239.394  299.812  INDEF  INDEF
239.505  303.484  INDEF  INDEF
760.526  321.709  14.770  0.267
288.233  341.016  13.778  0.105
295.068  357.660  12.372  0.030
753.547  379.485  13.325  0.068
755.767  401.980  14.476  0.195
459.530  411.354  14.160  0.138
94.762  455.335  13.767  0.102
316.625  462.059  14.226  0.150
940.450  461.986  11.483  0.012
588.065  493.487

INDEFなんてのもあります。これは、たまたまバッドピクセルが測光領域に含まれていた、あるいは、サチった星です。  
測光誤差を(例えば)0.2等以下のものだけに絞ることで、INDEFのものも削除できます。  

In [15]:
iraf.txdump('out1.mag', fields='xc,yc,mag,merr', expr='merr < 0.2', Stdout='result1.txt')

In [16]:
!head -n 10 result1.txt   #  一般的なlinuxコマンドは!をつけて実行可能 

312.046  30.898  13.601  0.085
785.510  63.492  11.574  0.014
640.120  75.714  14.393  0.178
644.934  105.562  14.427  0.187
250.493  113.488  11.479  0.013
344.480  116.890  12.707  0.040
603.322  119.941  12.490  0.034
222.478  129.444  12.646  0.038
630.258  176.196  14.349  0.173
713.843  188.084  13.186  0.058


### 測光値の較正  

ここで求まったものは、機械等級です。等級のゼロ点を適当に設定したものです。 
どうやって本当の等級に直せば良いでしょうか。  

実は、target1の真ん中の星は標準星といって、明るさのわかっている星です。  
Persson et al. (1998)のNo.9182の星で、Hバンドでの等級は H=11.082(0.010)です。  

この標準星の機械等級が10.301と求められたので、

In [17]:
11.082 - 10.301

0.7810000000000006

result1.txtの全ての星の等級に + 0.781足せば較正ができたことになります。  
ここで、さらっとnumpyを使っちゃいます。  

In [18]:
import numpy as np
np.set_printoptions(precision=3, suppress=True)  #  suppress=Trueで指数表示禁止

In [19]:
mlist = np.loadtxt('result1.txt')
mlist[:, 2] = mlist[:, 2] + 0.781 #  3列目全てに0.781を足す 
print (mlist)

[[  312.046    30.898    14.382     0.085]
 [  785.51     63.492    12.355     0.014]
 [  640.12     75.714    15.174     0.178]
 [  644.934   105.562    15.208     0.187]
 [  250.493   113.488    12.26      0.013]
 [  344.48    116.89     13.488     0.04 ]
 [  603.322   119.941    13.271     0.034]
 [  222.478   129.444    13.427     0.038]
 [  630.258   176.196    15.13      0.173]
 [  713.843   188.084    13.967     0.058]
 [  502.04    207.513    13.77      0.047]
 [  794.28    208.412    15.294     0.197]
 [  948.199   235.36     15.06      0.17 ]
 [  174.436   237.846    14.326     0.091]
 [   35.209   238.813    12.707     0.02 ]
 [   38.877   244.428    12.922     0.023]
 [  673.529   250.964    14.52      0.1  ]
 [  528.589   265.68     15.186     0.193]
 [  598.709   265.575    14.773     0.119]
 [  646.488   270.671    11.834     0.008]
 [  288.233   341.016    14.559     0.105]
 [  295.068   357.66     13.153     0.03 ]
 [  753.547   379.485    14.106     0.068]
 [  755.767

In [20]:
np.savetxt('result1calib.txt', mlist, fmt='%.3f')

### 演習3  

演習2で処理をしたtarget2のフレーム(例えばidfs0020.fits)で測光をしてみましょう。  
target2のフレームには標準星は写っていません。しかし、target1と近い時刻に観測をしているので、target1と同じ較正値が使えるものとしてください。


